crf测试pku中文语料第3版

改进：将训练集从[word,label]扩充成[word,cuttag,postag,label]

In [135]:
from sklearn.metrics import classification_report
from sklearn.preprocessing import LabelBinarizer
import pycrfsuite
import jieba.posseg as jbpos
from itertools import chain

In [136]:
sentList = []
f = open('./train_data', 'r', encoding='utf-8')
pair = []
for line in f:
    line = line.encode('utf-8').decode('utf-8-sig').strip()  # 去掉首字前的\ufeff
    if line:
        l = line.split(' ')
        pair.append([l[0], l[1]])
    else:
        sentList.append(pair)
        pair = []
f.close()

In [138]:
print(sentList[3])

[['去', 'N'], ['年', 'N'], ['，', 'N'], ['我', 'N'], ['们', 'N'], ['又', 'N'], ['被', 'N'], ['评', 'N'], ['为', 'N'], ['“', 'N'], ['北', 'B-LOC'], ['京', 'I-LOC'], ['市', 'I-LOC'], ['首', 'N'], ['届', 'N'], ['家', 'N'], ['庭', 'N'], ['藏', 'N'], ['书', 'N'], ['状', 'N'], ['元', 'N'], ['明', 'N'], ['星', 'N'], ['户', 'N'], ['”', 'N'], ['。', 'N']]


In [139]:
def get_sentence(token):  # 把[[word, '', '', ''], [word, '', '', '']...]组合成'洗衣机，...'
    sentence = ''
    for t in token:
        sentence += t[0]  # 取第一个字
    return sentence

In [140]:
def get_cut_and_seg(token):  # token格式为[[word, '', '', ''], [word, '', '', '']...]
    # print(get_sentence(token))  # 海钓比赛地点在厦门与金门之间的海域。
    wordlist = jbpos.cut(get_sentence(token))  # 海钓/nr 比赛/vn 地点/n 在/p 厦门/ns
    res = list()
    index = 0
    for w in wordlist:
        for i in range(len(w.word)):  # 考虑每个词语中的每个字
            if len(w.word) == 1:  # 如果词语的长度为1，即只有一个字，标志为S
                status = 'S'
            elif i == 0:  # 如果词语的长度不为1，则第一个字，标志为B
                status = 'B'
            elif i == len(w.word) - 1:  # 如果词语的长度不为1，则最后一个字，标志为E
                status = 'E'
            else:  # 如果词语的长度不为1，则除了首/尾之外的字，标志为I
                status = 'I'
            token[index][1] = status  # 把每个字的S/B/I/E标志，赋值给token[index][1]
            token[index][2] = w.flag  # 一个词语内的每个字的flag都是一样的
            index += 1
    return res

In [158]:
if __name__=='__main__':
    train_sents = []
    test_sents = []
    pairList = []
    train = open('./train_data', 'r', encoding='utf-8')
    test = open('./test_data', 'r', encoding='utf-8')
    for line in train :
        line = line.encode('utf-8').decode('utf-8-sig').strip()
        if line:
            l = line.split(' ')
            word = l[0]
            pos =l[1]
            pairList.append([word,'','', pos])
        else:
            get_cut_and_seg(pairList)
            train_sents.append(pairList)
            pairList = []
            
    for line in test :
        line = line.encode('utf-8').decode('utf-8-sig').strip()
        if line:
            l = line.split(' ')
            word = l[0]
            pos =l[1]
            pairList.append([word,'','', pos])
        else:
            get_cut_and_seg(pairList)
            test_sents.append(pairList)
            pairList = [] 
            
    train.close()
    test.close()

In [159]:
print(train_sents[-1])

[['“', 'S', 'x', 'N'], ['能', 'S', 'v', 'N'], ['选', 'B', 'v', 'N'], ['出', 'E', 'v', 'N'], ['一', 'B', 'm', 'N'], ['个', 'E', 'm', 'N'], ['好', 'S', 'a', 'N'], ['村', 'B', 'n', 'N'], ['长', 'E', 'n', 'N'], ['来', 'S', 'v', 'N'], ['，', 'S', 'x', 'N'], ['是', 'S', 'v', 'N'], ['我', 'B', 'r', 'N'], ['们', 'E', 'r', 'N'], ['老', 'B', 'n', 'N'], ['百', 'I', 'n', 'N'], ['姓', 'E', 'n', 'N'], ['的', 'S', 'uj', 'N'], ['福', 'B', 'n', 'N'], ['分', 'E', 'n', 'N'], ['！', 'S', 'x', 'N'], ['”', 'S', 'x', 'N'], ['记', 'B', 'n', 'N'], ['票', 'E', 'n', 'N'], ['板', 'S', 'ng', 'N'], ['上', 'S', 'f', 'N'], ['出', 'B', 'v', 'N'], ['现', 'E', 'v', 'N'], ['了', 'S', 'ul', 'N'], ['左', 'S', 'm', 'B-PER'], ['金', 'B', 'nz', 'I-PER'], ['文', 'E', 'nz', 'I-PER'], ['的', 'S', 'uj', 'N'], ['名', 'B', 'n', 'N'], ['字', 'E', 'n', 'N'], ['，', 'S', 'x', 'N'], ['而', 'B', 'c', 'N'], ['且', 'E', 'c', 'N'], ['票', 'B', 'n', 'N'], ['数', 'E', 'n', 'N'], ['扶', 'B', 'i', 'N'], ['摇', 'I', 'i', 'N'], ['直', 'I', 'i', 'N'], ['上', 'E', 'i', 'N'], ['，', 'S', 'x

In [160]:
print(test_sents[-1])

[['一', 'B', 'm', 'N'], ['个', 'E', 'm', 'N'], ['名', 'B', 'v', 'N'], ['叫', 'E', 'v', 'N'], ['左', 'S', 'm', 'B-PER'], ['金', 'B', 'nz', 'I-PER'], ['文', 'E', 'nz', 'I-PER'], ['的', 'S', 'uj', 'N'], ['村', 'B', 'n', 'N'], ['民', 'E', 'n', 'N'], ['竟', 'S', 'd', 'N'], ['自', 'B', 'r', 'N'], ['己', 'E', 'r', 'N'], ['投', 'S', 'v', 'N'], ['了', 'S', 'ul', 'N'], ['自', 'B', 'r', 'N'], ['己', 'E', 'r', 'N'], ['一', 'B', 'm', 'N'], ['票', 'E', 'm', 'N'], ['，', 'S', 'x', 'N'], ['而', 'B', 'c', 'N'], ['且', 'E', 'c', 'N'], ['主', 'B', 'b', 'N'], ['动', 'E', 'b', 'N'], ['向', 'S', 'p', 'N'], ['周', 'B', 'f', 'N'], ['围', 'E', 'f', 'N'], ['的', 'S', 'uj', 'N'], ['村', 'B', 'n', 'N'], ['民', 'E', 'n', 'N'], ['畅', 'B', 'v', 'N'], ['述', 'E', 'v', 'N'], ['他', 'S', 'r', 'N'], ['的', 'S', 'uj', 'N'], ['想', 'B', 'v', 'N'], ['法', 'E', 'v', 'N'], ['：', 'S', 'x', 'N'], ['“', 'S', 'x', 'N'], ['我', 'S', 'r', 'N'], ['竞', 'B', 'vn', 'N'], ['选', 'E', 'vn', 'N'], ['村', 'B', 'n', 'N'], ['长', 'E', 'n', 'N'], ['的', 'S', 'uj', 'N'], ['意', 'B',

In [161]:
def word2features(sent, i):
    word = sent[i][0]
    cuttag = sent[i][1]
    postag = sent[i][2]
    features = [
        'bias',
        'word=' + word,
        'word.isdigit=%s' % word.isdigit(),
        'postag=' + postag,
        'cuttag=' + cuttag,
    ]
    if i > 0:
        word1 = sent[i - 1][0]
        postag1 = sent[i - 1][2]
        cuttag1 = sent[i - 1][1]
        features.extend([
            '-1:word=' + word1,
            '-1:postag=' + postag1,
            '-1:cuttag=' + cuttag1,
        ])
    else:
        features.append('BOS')

    if i < len(sent) - 1:
        word1 = sent[i + 1][0]
        postag1 = sent[i + 1][2]
        cuttag1 = sent[i + 1][1]
        features.extend([
            '+1:word=' + word1,
            '+1:postag=' + postag1,
            '+1:cuttag=' + cuttag1,
        ])
    else:
        features.append('EOS')

    return features

In [162]:
def sent2features(sent):
    return [word2features(sent, i) for i in range(len(sent))]


def sent2labels(sent):
    return [label for token, cuttag, postag, label in sent]


def sent2tokens(sent):
    return [token for token, cuttag, postag, label in sent]

In [163]:
print(sent2features(train_sents[0])[0])

['bias', 'word=藏', 'word.isdigit=False', 'postag=nz', 'cuttag=B', 'BOS', '+1:word=书', '+1:postag=nz', '+1:cuttag=E']


In [164]:
print(sent2labels(train_sents[0])[2])

N


In [165]:
X_train = [sent2features(s) for s in train_sents]  # 不加[]是generator类型，不会输出。加了[]，可直接输出加了[]之后的结果。
y_train = [sent2labels(s) for s in train_sents]

X_test = [sent2features(s) for s in test_sents]  # 格式 [[[每个word的特征],[每个word的特征]],（到此处是一个s）,[(每个word的特征)],...]，
y_test = [sent2labels(s) for s in test_sents] 

In [166]:
print(len(y_test))

6524


In [167]:
print(y_test[1111])

['B-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'I-ORG', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N', 'N']


In [168]:
trainer = pycrfsuite.Trainer(verbose=False)

In [169]:
for xseq, yseq in zip(X_train, y_train): 
    trainer.append(xseq, yseq) 

In [170]:
# trainer的设置参数
trainer.set_params({
    'c1':1.0,
    'c2':1e-3,
    'max_iterations':50,
    'feature.possible_transitions':True
})

In [171]:
trainer.train('conll2002_3.crfsuite')

In [172]:
tagger = pycrfsuite.Tagger()
tagger.open('conll2002_3.crfsuite')

In [173]:
print('ture:', ' '.join(sent2labels(test_sents[17])))
print('\n')
print('predicted:', ' '.join(tagger.tag(sent2features(test_sents[17]))))

ture: N N N N N N N N N N N N N N N N N N N N N N N N N N N


predicted: N N N N N N N N N N N N N N N N N N N N N N N N N N N


In [176]:
def bio_classification_report(y_true, y_pred):
    '''
    Classification report for a list of BIO-encoded sequences.
    It computes token-level metrics and discards "O" labels.
    '''
    lb = LabelBinarizer()
    y_true_combined = lb.fit_transform(list(chain.from_iterable(y_true)))
    y_pred_combined = lb.transform(list(chain.from_iterable(y_pred)))
    tagset = set(lb.classes_) - {'N'} 
    tagset = sorted(tagset, key=lambda tag: tag.split('-', 1)[::-1])
    class_indices = {cls: idx for idx, cls in enumerate(lb.classes_)} 

    return classification_report(
        y_true_combined,
        y_pred_combined,
        labels=[class_indices[cls] for cls in tagset],
        target_names=tagset,
    )

In [177]:
y_pred = [tagger.tag(xseq) for xseq in X_test]

print(bio_classification_report(y_test, y_pred))

             precision    recall  f1-score   support

      B-LOC       0.88      0.83      0.85      5391
      I-LOC       0.83      0.82      0.83      7214
      B-ORG       0.85      0.70      0.77      3082
      I-ORG       0.86      0.75      0.80     12638
      B-PER       0.91      0.81      0.86      2760
      I-PER       0.91      0.86      0.88      5463

avg / total       0.87      0.79      0.83     36548



In [63]:
print('predicted:', ' '.join(tagger.tag('我爱北京天安门')))

predicted: B-LOC I-LOC I-LOC I-LOC I-LOC I-LOC I-LOC


In [70]:
def split_by_words(sentence):
    res = list()
    for word in sentence:
        res.append([word, '', '', ''])
    get_cut_and_seg(res)
    return res

In [71]:
def get_type_encode(text):
    if text.__contains__('PRO'):
        return 'product_name'
    elif text.__contains__('PER'):
        return 'person_name'
    elif text.__contains__('TIM'):
        return 'time'
    elif text.__contains__('ORG'):
        return 'org_name'
    elif text.__contains__('LOC'):
        return 'location'
    else:
        return 'unknown'

In [76]:
def format_boson_data_encode(text, tag):
    res = ""
    status = 0
    for i in range(len(text)):
        if status == 0 and tag[i] == 'N':  # 第一个字的标签
            res += text[i]  # 不是要识别的字，加入到res中
        elif status == 0 and tag[i] != 'N':  # 第一个字的标签是要识别的字
            status = 1
            res += "{{" + get_type_encode(tag[i]) + ":" + text[i]
        elif status == 1 and str(tag[i]).startswith('I'):  # 是要识别的字，而且是词中或词尾字
            res += text[i]
        elif status == 1:
            res += "}}"
            if tag[i] == 'N':
                status = 0
                res += text[i]
            else:
                status = 1
                res += "{{" + get_type_encode(tag[i]) + ":" + text[i]
    return res

In [77]:
def ner(text):
    tagger = pycrfsuite.Tagger()
    tagger.open('conll2002_3.crfsuite')
    sent = split_by_words(text)
    tag_result = tagger.tag(sent2features(sent))
    text_result = format_boson_data_encode(text, tag_result)
    return text_result

In [78]:
print('predicted:', ner('我站在北京的天安门门口'))

predicted: 我站在{{location:北京}}的{{location:天安门}}门口
